In [42]:
from IPython.core.debugger import set_trace

class Computer:
    
    def __init__(self, state, inputs=None):
        self.state = state
        self.inputs = inputs or []
        self.outputs = []
        self.running = True
        self.i = 0
        self.out_log = []

    def add(self, s, i, d):
#         print(f"add - {s[i:i+4]}")
        a = s[i+1] if d.get(0, False) else s[s[i+1]]
        b = s[i+2] if d.get(1, False) else  s[s[i+2]]
        s[s[i+3]] = a + b
        return s, i+4, True
    
    def multiply(self, s, i, d):
#         print(f"mult - {s[i:i+4]}")
        a = s[i+1] if d.get(0, False) else s[s[i+1]]
        b = s[i+2] if d.get(1, False) else  s[s[i+2]]
        s[s[i+3]] = a * b
        return s, i+4, True
    
    def val_in(self, s, i, d):
#         print(f"in - {s[i:i+2]}")
        if not self.inputs:
#             print("No inputs available! Waiting...")
            return s, i, True
        x = self.inputs.pop(0)
        s[s[i+1]] = x
        return s, i+2, True
    
    def val_out(self, s, i, d):
        o = s[i+1] if d.get(0, False) else s[s[i+1]]
#         print(f"output! {o} - {s[i:i+2]}")
        self.outputs.append(o)
        self.out_log.append(o)
        return s, i+2, True
    
    def jump_true(self, s, i, d):
#         print(f"jump_true - {s[i:i+3]}")
        a = s[i+1] if d.get(0, False) else s[s[i+1]]
        b = s[i+2] if d.get(1, False) else  s[s[i+2]]
        return s, i+3 if a == 0 else b, True
        
    def jump_false(self, s, i, d):
#         print(f"jump_false - {s[i:i+3]}")
        a = s[i+1] if d.get(0, False) else s[s[i+1]]
        b = s[i+2] if d.get(1, False) else  s[s[i+2]]
        return s, b if a == 0 else i+3, True
        
    def less_than(self, s, i, d):
#         print(f"less_than - {s[i:i+4]}")
        a = s[i+1] if d.get(0, False) else s[s[i+1]]
        b = s[i+2] if d.get(1, False) else  s[s[i+2]]
        s[s[i+3]] = 1 if a < b else 0
        return s, i+4, True
        
    def equals(self, s, i, d):
#         print(f"equals - {s[i:i+4]}")
        a = s[i+1] if d.get(0, False) else s[s[i+1]]
        b = s[i+2] if d.get(1, False) else  s[s[i+2]]
        s[s[i+3]] = 1 if a == b else 0
        return s, i+4, True
        
    def kill(self, s, i, d):
#         print("KILL")
        self.outputs.append("KILL")
        return s, i, False
                              
    def fail(self, se, s, i, d):
        print(f"Unknown operation: {s[i:i+4]} Exiting...")
        return s, i, False
    
    operations = {
        1: add,
        2: multiply,
        3: val_in,
        4: val_out,
        5: jump_true,
        6: jump_false,
        7: less_than,
        8: equals,
        99: kill
    }
    
    def tick(self):
        op = int(str(self.state[self.i])[-2:])
        directs = {i:x for i, x in enumerate(str(self.state[self.i])[:-2][::-1]) if x == '1'}
        self.state, self.i, self.running = self.operations.get(op, self.fail)(self, self.state, self.i, directs)
    
    def run_program(self):
        while(self.running):
            self.tick()
        return self.state
    